In [ ]:
import os
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
from google.protobuf import message_factory

In [ ]:
loc=r'C:\Users\hp\TEST'

In [ ]:
df = pd.DataFrame(columns=['points'])

In [ ]:
mp_pose = mp.solutions.pose

# Create DataFrame to store features and labels
features_df = pd.DataFrame(columns=['feature'])
yoga_pose_labels = []
image_root_dir=r'C:\Users\hp\TEST\image_root_dir'
# List all directories (poses) in the image root directory
pose_directories = os.listdir(image_root_dir)

# Counter for DataFrame row index
count = 0

# Process each pose directory
for pose_name in pose_directories:
    pose_dir_path = os.path.join(image_root_dir, pose_name)
    if not os.path.isdir(pose_dir_path):
        continue
    
    # List all images in the current pose directory
    image_files = os.listdir(pose_dir_path)
    
    for image_name in image_files:
        image_path = os.path.join(pose_dir_path, image_name)
        with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
            frame = cv2.imread(image_path)
            if frame is None:
                print(f"Error: Image at {image_path} could not be read.")
                continue
            
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = pose.process(image)
            image.flags.writeable = True

            if results.pose_landmarks:
                try:
                    landmarks = results.pose_landmarks.landmark
                    
                    # Extract required landmark coordinates
                    feature = np.array([
                        landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y,
                        landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y,
                        landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y,
                        landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y,
                        landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y,
                        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,
                        landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y,
                        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y,
                        landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y,
                        landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y,
                        landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y,
                        landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y
                    ])
                    
                    # Append feature and label to the DataFrame
                    features_df.loc[count] = [feature]
                    yoga_pose_labels.append(pose_name)
                    count += 1
                
                except Exception as e:
                    print(f"Error processing image {image_path}: {e}")
            else:
                print(f"No landmarks detected for image {image_path}.")

# Create DataFrame for labels
labels_df = pd.DataFrame(yoga_pose_labels, columns=['label'])

# Concatenate features and labels DataFrames
final_df = pd.concat([features_df, labels_df], axis=1)

# Save the DataFrame to a CSV file
final_df.to_csv("yoga_pose_data.csv", index=False)

# Display the first few rows of the DataFrame
print(final_df.head())

In [ ]:
df=pd.DataFrame(df['points'].values.tolist())

In [ ]:
yoga_pose

In [ ]:
label_df = pd.DataFrame (yoga_pose, columns=['label'])
df = pd.concat ([df, label_df], axis=1)

In [ ]:
df.head()

In [ ]:
df.to_csv("yoga_pose_data.csv",index=False)

## Creating model

In [ ]:
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
df=pd.read_csv("yoga_pose_data.csv")

In [ ]:
df.head()

In [ ]:
lb = LabelEncoder()
y = lb.fit_transform(df['label'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['label'],axis=1), y, test_size=0.1, shuffle=True, random_state=2)

In [ ]:
from sklearn import preprocessing 
import sklearn
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
from sklearn import svm
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model = svm.SVC(kernel='linear')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
model=GaussianNB()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
model = DecisionTreeClassifier(criterion = "entropy", random_state = 100,max_depth = 3, min_samples_leaf = 5)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
model = RandomForestClassifier(bootstrap=False,max_depth=8,max_features='sqrt',min_samples_leaf=1,min_samples_split=2,n_estimators=100)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
import pickle

In [ ]:
best=0
for _ in range(100):
    X_train, X_test, y_train, y_test = train_test_split(df.drop(['label'],axis=1), y, test_size=0.2, shuffle=True, random_state=2)
    model = RandomForestClassifier(bootstrap=False,max_depth=8,max_features='sqrt',min_samples_leaf=1,min_samples_split=2,n_estimators=100)
    model.fit(X_train, y_train)
    acc = model.score(X_test, y_test)
    if acc > best:
        best = acc
        with open("studentmodel.pickel", "wb") as f:
            pickle.dump(model, f)
print(best)

In [ ]:
pickle_in = open("studentmodel.pickel", "rb")
model = pickle.load(pickle_in)

In [ ]:
predictions = model.predict(X_test)

In [ ]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [ ]:
def calculate_angle(a,b,c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [ ]:
def start_yoga():
    cap = cv2.VideoCapture(0)
    count = 0
    frameloc=[]
    pred=''
    count1=0
    count1=0
    count1=0
    count1=0
    count1=0
    t=200
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
            while cap.isOpened():
            
                ret, frame = cap.read()

                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                image.flags.writeable = False
                results = pose.process(image)
                image.flags.writeable = True
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

                try:
                    landmarks = results.pose_landmarks.landmark

               

                    a = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
                    b = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                    c = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                    d = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                    e = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                    f = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                
                    g = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
                    h = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                    i = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
                    j = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                    k = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
                    l = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                    if landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].visibility>0.5:
                        feature = np.concatenate((a,b,c,d,e,f,g,h,i,j,k,l))
                        frameloc.append(feature)
                        angle= calculate_angle(h,j,l)
                        count+=1
                        if count>3:
                            predictions = model.predict(frameloc)
                            frameloc=[]
                            count=0
                            predictions=list(predictions)
                            if predictions.count(3)==4 and calculate_angle(h,j,l)<100:
                                    pred='tree'
                                    count1+=1
                            elif predictions.count(4)==4 and calculate_angle(h,j,l)<150 and calculate_angle(g,i,k)<150 and count1==t:
                                    pred='goddess'
                                    count2+=1
                            elif predictions.count(2)==4 and count2==t :
                                    pred='plank'
                                    count3+=1
                            elif predictions.count(4)==4 and calculate_angle(g,i,k)<150 and calculate_angle(h,j,l)>165 and count3==t:
                                    pred='warrior'
                                    count4+=1
                            elif predictions.count(0)==4 and calculate_angle(i,g,e)<150 and count4==t:
                                    pred='downdog'
                                    count5+=1
                                
                except:
                    pass                

            
            

                #cv2.putText(image, 'Count', (15,12),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, str(pred), (10,60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
                pred=''
                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)               

                cv2.imshow('Frame', image)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    cap.release()
                    cv2.destroyAllWindows()
                    break
                if count5==t:
                    cap.release()
                    cv2.destroyAllWindows()
                    break

In [ ]:
cap.release()
cv2.destroyAllWindows()

In [ ]:
x=start_yoga()